In [1]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy


In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6



Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-03


In [3]:
#Solver
model.solver = 'cplex'

In [4]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
mm.read_medium_csv('../misc/photo_medium.csv', model)

#Test natin kung magwowork sa est. infinite
#Yes setting bounds to 1000 works
model_media = model.medium
model.medium

{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000,
 'EX_no3(e)': 1000000,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_nh4(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

In [5]:
# #Add constraints to model

# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

BS_photon_import = model.reactions.PRISM_blue_LED_BS
M_photon_import = model.reactions.PRISM_blue_LED_M

photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
                                       - BS_photon_import.flux_expression,
                                       lb = 0, ub = 1000)
model.add_cons_vars(photon_flux)


co2tex_m = model.reactions.CO2tex_M
co2tex_bs = model.reactions.CO2tex_BS

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
#Lower bound based  on Dark CO2 uptake rates from Ermakova et al (2020)
co2_import_cons = model.problem.Constraint(co2tex_m.flux_expression 
                                           + co2tex_bs.flux_expression, lb = -inf, ub = 29)
model.add_cons_vars(co2_import_cons)

#CO2 BS intake must be limited owing to its position in the leaf tissue

#model.add_cons_vars(model.problem.Constraint(co2tex_bs.flux_expression, lb = 0, ub = 2))
# #Note: CO2 uptake is not restricted in BS cells but will be constrained to be  equal/less than M cell

# co2_flux_cons = model.problem.Constraint(BS_co2_import.flux_expression - M_co2_import.flux_expression, lb = 0, ub = inf)
# model.add_cons_vars(co2_flux_cons)

# #Oks na

# #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# Values from Von Caemmerer (2000) in mixed C3-C4 photosynthetic plants is 1 umol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
# #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# #I'll assume that the ratio is at 15:1 in the meantime, meaning that there is minimal gas exchange into the BS cell
co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:20, co2tex_bs.id:1},model)
model.add_cons_vars(co2_ratio_cons)

# #Automatic na palang nalalagay siya as constraint

# #First is the BS and M Cell constraints

In [ ]:
# #What if I restrict yung import ng O2?

# #Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
# #Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
# o2tex_m = model.reactions.O2tex_M
# o2tex_bs = model.reactions.O2tex_BS
# o2_import_cons = model.problem.Constraint(o2tex_bs.flux_expression, lb = -inf, ub = inf)
# model.add_cons_vars(o2_import_cons)


In [54]:
# #Add enzyme constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0.

# #PEPC
wt_pepc = 2.4 #umol m-2 s-1
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 15.74 #umol m-2  s-1 bar-1

#PEPC constraint (Reaction id: PPCc)
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = wt_pepc)
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKS) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected
ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M

wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
mdh2c_M = model.reactions.MDH2c_M
mdh2c_BS = model.reactions.MDH2c_BS
mdh2s_M = model.reactions.MDH2s_M
mdh2s_BS = model.reactions.MDH2s_BS

wt_mdh_cons = model.problem.Constraint(mdh2c_M.flux_expression 
                                       + mdh2c_BS.flux_expression 
                                       + mdh2s_M.flux_expression 
                                       + mdh2s_BS.flux_expression, 
                                       lb= 0, ub=wt_mdh)
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
#Let's see if I can force it to have a small amount of flux 
nadp_me_M = model.reactions.MDH2s_M
nadp_me_BS = model.reactions.MDH2s_BS

wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                         + nadp_me_BS.flux_expression,
                                         lb= 0, ub=wt_nadp_me)
model.add_cons_vars(wt_nadpme_cons)
#Weird, it causes the activity of RBCL to significantly vary from the actual values


#I should add constraints for Carbonic Anhydrase
#I should constrain it to 0.4 ubar, which would constitute ambient partial pressure
#
#HCO3Es
#HCO3Ec
#HCO3Em
#Flux is negative so  it needs to go backwards


hco3es_m = model.reactions.HCO3Es_M.flux_expression
hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
hco3em_m = model.reactions.HCO3Em_M.flux_expression
hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                   + hco3es_bs + hco3ec_bs + hco3em_bs,
                                  lb = -wt_CA, ub = 0)
model.add_cons_vars(ca_cons)
#Rbcl constaints

In [7]:
#Turn off old photorespiratory reaction from the ios2164 model
model.reactions.RBPC2s_M.bounds = (0,0)
model.reactions.RBPC2s_BS.bounds = (0,0)

#Turn off H2O transporter in m cell
model.reactions.H2Otex_M.bounds = (0,0)


In [8]:
#Let's try to add Rubisco flux ratio

rbpc_M = model.reactions.RBPCs_M.flux_expression
rbpc_BS = model.reactions.RBPCs_BS.flux_expression
rbpo_M = model.reactions.RBPOs_M.flux_expression
rbpo_BS = model.reactions.RBPOs_BS.flux_expression

#Constraint such that it is limited to 132 umol m-2 s-1
rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)

model.add_cons_vars(rbcl_vcmax_cons)

In [44]:
#Fixed ratio format (r0:value0, r1:value1)
#(value0*r1_fe - value1*r0_fe)
#Constraints for rbcl flux

rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                     - 1*(rbpc_M + rbpc_BS),
                                     lb=0,ub=1000)

# model.add_cons_vars(rbcl_vcvo)
#What if I simply constrained that of the M cell one to 3:1?

In [91]:
#Modifications to medium

model_media['EX_photonVis(e)'] = 800#PPFD used in Ermakova et al for high light

#Set CO2 bounds to ~29 m-2 s-1
model_media['EX_co2(e)'] = 400 #mbar
model_media['EX_o2(e)'] = 200000 #mbar

#Test if heterotrophic conditions fix Biomass
#model_media['EX_sucr(e)_M'] = inf

#Nope, still doesn't produce biomass.
#How about Protons?
model_media['EX_h(e)'] = 0
#What if I constrained nitrogen uptake?
#
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = 0
model_media['EX_h2o(e)'] = inf
model.medium = model_media



In [102]:

with model:
    model.add_cons_vars(rbcl_vcvo)
    sample_fluxes = cobra.flux_analysis.pfba(model)
model.summary(fva=0.90)

#Weird, no flux is  being produced right now.


Metabolite,Reaction,Flux,Range,C-Number,C-Flux
co2_e,EX_co2(e),29,[0; 29],1,100.00%
h2s_e,EX_h2s(e),0.02707,[0; 0.6844],0,0.00%
no3_e,EX_no3(e),1.592,[1.433; 45.43],0,0.00%
o2_e,EX_o2(e),16.72,[2.903; 414.7],0,0.00%
photonVis_e,EX_photonVis(e),800,[0; 800],0,0.00%
pi_e,EX_pi(e),0.5124,[0.4612; 99.1],0,0.00%
Metabolite,Reaction,Flux,Range,C-Number,C-Flux
ltlnnp_c1,DM<NNP(c)_BS,0,[-0.4981; 0],27,0.00%
ltlnnp_c0,DM<NNP(c)_M,0,[-0.4981; 0],27,0.00%
3-non_s1,DM_3-NON(s)_BS,0,[-0.9778; 0],9,0.00%


In [103]:
# This is cool, the M cell is similar to the WT rate of 83.59+-5.63 :)
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC M', sample_fluxes['PPCc_BS'])
print('NADP-ME M', sample_fluxes['MDH2s_M'], 'NADP-ME BS', sample_fluxes['MDH2s_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_rescaled_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_rescaled_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])
print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])

rbcl M cell:  93.97313940253744 rbcl BS cell:  38.02686059746172
rbcl M cell (photorespiration) 14.29206916716673 rbcl BS cell (PR) 29.707930832832993
PEPC M 1.8817735176160095 PEPC M 0.5182264823839904
NADP-ME M 0.0 NADP-ME BS 0.0
Biomass M:  0.0 Biomass BS 0.0
co2 consumption M 0.0 co2 consumption BS 0.0
Photosystem II M 28.58413833433346 PSII BS 46.4306797426793
PSI M 111.67615678464237 PSI BS 97.74661483975201


In [105]:
#Check first the Carbonic anhydrases
model.metabolites.o2_c0.summary(fva=0.95)


Percent,Flux,Range,Reaction,Definition
30.04%,15.22,[-991.4; 1000],O2tex_M,o2_e <=> o2_c0
69.96%,35.45,[-194.3; 75.25],O2tsr_M,o2_c0 <=> o2_s0
0.00%,0,[-0.55; 0.5283],P450MOc_M,fad_c0 + h2o_c0 + t-ztn_c0 <=> fadh2_c0 + ipadne_c0 + o2_c0
Percent,Flux,Range,Reaction,Definition
0.00%,0,[-0.2444; 0],22HCAMHc_M,5camp3e_c0 + h_c0 + nadph_c0 + o2_c0 --> 22h5camp3e_c0 + h2o_c0 + nadp_c0
0.00%,0,[-0.2357; 0],34HPPORc_M,34hpp_c0 + o2_c0 --> co2_c0 + hgentis_c0
0.00%,0,[-3.3; 0],ACDOc_M,12dmpo_c0 + o2_c0 --> 2kmb_c0 + for_c0 + h_c0
0.00%,0,[-2.2; 0],AMCPCOc_M,1acpc_c0 + ascb-L_c0 + h_c0 + o2_c0 --> co2_c0 + cyan_c0 + dhdascb_c0 + ethylene_c0 + 2.0 h2o_c0
0.00%,0,[-0.4552; 0],APG3Hc_M,apgn_c0 + h_c0 + nadph_c0 + o2_c0 --> h2o_c0 + ltln_c0 + nadp_c0
0.00%,0,[-4.4; 0],ARG2MO_M,arg-L_c0 + o2_c0 --> co2_c0 + gabut_c0 + h2o_c0


In [ ]:
sample_fluxes = cobra.flux_analysis.pfba(model)
pd_rxns = [rxns for rxns in model.reactions if "_pd" in rxns.id]

for items in pd_rxns:
    print(items.id, sample_fluxes[items.id])

In [ ]:
for items in model.reactions:
    print(items.id, sample_fluxes[items.id])

In [ ]:
#Note:
#Unconstrained CO2 intake leads to a saturated  net CO2 assimilation rate of around 40 umol m-2 s-1
#Better to constrain this to known uptake rates instead

In [ ]:
#Oh wtf it works, below 500 ppfd no net intake below dark respiration is observed tapos nasasaturate siya agad to  the max value
#Cooooool. Based on Weber et al (1986) the critical point is at around 500-600 umol he m-2 s-1
#Weber, J. A., Tenhunen, J. D., Gates, D. M., & Lange, O. L. (1987). Effect of photosynthetic photon flux density on carboxylation efficiency. Plant physiology, 85(1), 109-114.


In [ ]:
#What I've done was unconstrained CO2 exchange to allow for CO2 efflux. At around 300-400 ppfd a net release of CO2 is observed from the mdoel, consistent 
#with Nobel (2020)'s  'Leaves and Fluxes'
#I need to read more on this. This is actually quite interesting


In [ ]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.